# TP 3 (17Co2024)
#### Integrantes
- Adassus, Luciano
- de Pedro Mermier, Ignacio
- Cagua, Jonathan

## Parte 1

Obtener una detección del logo en cada imagen sin falsos positivos

In [1]:
%matplotlib qt
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Cargo las imagenes

In [2]:

template = cv.imread('template/pattern.png')

coca_logo_1 = cv.imread('images/coca_logo_1.png')
coca_logo_2 = cv.imread('images/coca_logo_2.png')

coca_cola_logo = cv.imread('images/COCA-COLA-LOGO.jpg')

logo_1 = cv.imread('images/logo_1.png')

coca_multi = cv.imread('images/coca_multi.png')

coca_retro_1 = cv.imread('images/coca_retro_1.png')
coca_retro_2 = cv.imread('images/coca_retro_2.png')
 

Analizo el tamaño de cada imagen

In [3]:
# Analizo los tamaños de las imagenes
print('Shapes')
print('coca_logo_1',coca_logo_1.shape)
print('coca_logo_2',coca_logo_2.shape)
print('coca_cola_logo',coca_cola_logo.shape)
print('logo_1',logo_1.shape)
print('coca_multi',coca_multi.shape)
print('coca_retro_1',coca_retro_1.shape)
print('coca_retro_2',coca_retro_2.shape)

Shapes
coca_logo_1 (500, 207, 3)
coca_logo_2 (363, 233, 3)
coca_cola_logo (1389, 1389, 3)
logo_1 (450, 687, 3)
coca_multi (598, 799, 3)
coca_retro_1 (493, 715, 3)
coca_retro_2 (429, 715, 3)


In [4]:
# Funcion para detectar template matching con multiples escalas

def multiScaleTemplateMatching(template,image,edge_detector="canny",matchTempMethod = cv.TM_CCOEFF_NORMED,n_scales = 12):
    # edge_detector puede ser Canny o Laplacian
    if edge_detector != "canny" and edge_detector != "laplacian":
        print('El parametro edge detector debe ser "canny" o "laplacian"')
        return
    # Paso las imagenes a escala de grises
    imgGray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    tmpGray = cv.cvtColor(template,cv.COLOR_BGR2GRAY)

    # Aplico Gaussian Blur para sacar ruido
    imgGray = cv.GaussianBlur(imgGray,(5,5),0)
    #Obtengo bordes segun el metodo de deteccion
    if edge_detector == "canny":
        imgGray = cv.Canny(imgGray, 100, 120, apertureSize = 3)
    elif edge_detector == "laplacian":
        imgGray = cv.Laplacian(imgGray, cv.CV_8U)
    
    # Voy a reescalar el template n_scales veces partiendo desde el ancho de la imagen
    th,tw = tmpGray.shape
    ih,iw = imgGray.shape
    scales = np.linspace(0, iw/tw, n_scales+1)[1:]

    # Listas para resultados
    matching_results = list()
    minmax_results = list()
    template_shapes = list()
    
    for scale in scales:
        # escalo el template
        tmpGray_scaled = cv.resize(tmpGray, None, fx=scale, fy=scale)
        # le aplico el Gaussian Blur para sacar ruido
        tmpGray_scaled = cv.GaussianBlur(tmpGray_scaled,(3,3),0)
        
        #Obtengo bordes segun el metodo de deteccion
        if edge_detector == "canny":
            tmpGray_scaled = cv.Canny(tmpGray_scaled, 150, 180, apertureSize = 3)
        elif edge_detector == "laplacian":
            tmpGray_scaled = cv.Laplacian(tmpGray_scaled, cv.CV_8U)
        
        # cv.imshow('tmpGray-scaled',tmpGray_scaled)
        # cv.waitKey(0)
        
        tsh,tsw = tmpGray_scaled.shape
        
        # Aplico el template matching 
        result = cv.matchTemplate(imgGray, tmpGray_scaled, matchTempMethod)
        # Normalizo el resultado 
        result = (result-np.mean(result))/np.std(result)
        
        # Guardo el resultado
        matching_results.append(result)

        # minMaxLoc para bounding box 
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
        
        #Guardo minMaxLocResult
        minmax_results.append([min_val, max_val, min_loc, max_loc])

        # Guardo shape del template
        template_shapes.append([tsh,tsw])

    # Obtengo el mejor matching para todos los casos y obtengo su ubicacion

    if matchTempMethod == cv.TM_CCOEFF_NORMED or matchTempMethod == cv.TM_CCORR_NORMED:
        # busco el maximo de maximos
        max_column = [sublist[1] for sublist in minmax_results]
        max_value = max(max_column)
        max_index = max_column.index(max_value)

        match_value = max_value
        match_index = max_index
        # Nivel de confianza de la deteccion (como estoy trabajando con la mejor deteccion le pongo 100%)
        confidence_value = 1
        
    elif matchTempMethod == cv.TM_SQDIFF_NORMED:
        # busco el minimo de minimos
        min_column = [sublist[0] for sublist in minmax_results]
        min_value = max(min_column)
        min_index = min_column.index(min_value)

        match_value = min_value
        match_index = min_index
        # Nivel de confianza de la deteccion (como estoy trabajando con la mejor deteccion le pongo 100%)
        confidence_value = 1

    else:
        print('matchTempMethod error')
        return
    
    # Imagen + Bounding box + Score
    matchImg = image.copy()
    topLeft = minmax_results[match_index][3]
    bottomRight = (minmax_results[match_index][3][0]+template_shapes[match_index][1],minmax_results[match_index][3][1]+template_shapes[match_index][0])
 
    cv.rectangle(matchImg, topLeft, bottomRight, (255,0,0), 2)
    cv.putText(matchImg,f'Score {str(int(confidence_value*100))}%', (topLeft[0],topLeft[1]-6), cv.QT_FONT_NORMAL, 0.45, (255,0,0))  
  
    return matchImg


Probamos las distintas imagenes con el detector de bordes "Laplaciano" y metodo de detección "Coeficiente de correlacion normalizado"

In [5]:
# Template Matching usando TM_CCOEFF_NORMED
filter_border = "laplacian" #"canny"
resultImg = multiScaleTemplateMatching(template, coca_logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_logo_2,filter_border,cv.TM_CCOEFF_NORMED, 1)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_cola_logo, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_1, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_2, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_multi, filter_border,cv.TM_CCOEFF_NORMED,39)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()


Probamos las distintas imagenes con el detector de bordes "Canny" y metodo de detección "Coeficiente de correlacion normalizado"

In [6]:
# Template Matching usando TM_CCOEFF_NORMED
filter_border = "canny" #"canny"
resultImg = multiScaleTemplateMatching(template, coca_logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_logo_2,filter_border,cv.TM_CCOEFF_NORMED, 1)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_cola_logo, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, logo_1, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_1, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_2, filter_border,cv.TM_CCOEFF_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_multi, filter_border,cv.TM_CCOEFF_NORMED,39)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()


Probamos las distintas imagenes con el detector de bordes "Laplaciano" y metodo de detección "Correlacion Cruzada normalizada"

In [7]:
# Template Matching usando TM_CCORR_NORMED
edge_detection = "laplacian" #"canny"
resultImg = multiScaleTemplateMatching(template, coca_logo_1, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_logo_2,edge_detection,cv.TM_CCORR_NORMED, 1)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_cola_logo, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, logo_1, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_1, edge_detection,cv.TM_CCORR_NORMED,12)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_2, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_multi, edge_detection,cv.TM_CCORR_NORMED,39)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()

Probamos las distintas imagenes con el detector de bordes "Canny" y metodo de detección "Correlacion Cruzada normalizada"

In [8]:
# Template Matching usando TM_CCORR_NORMED
edge_detection = "canny" #"canny"
resultImg = multiScaleTemplateMatching(template, coca_logo_1, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_logo_2,edge_detection,cv.TM_CCORR_NORMED, 1)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_cola_logo, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, logo_1, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_1, edge_detection,cv.TM_CCORR_NORMED,12)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_retro_2, edge_detection,cv.TM_CCORR_NORMED,10)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatching(template, coca_multi, edge_detection,cv.TM_CCORR_NORMED,39)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()

**Nota: Los valores de n_scales para los casos anteriores se encontraron de manera iterativa para cada uno de los casos**

## Parte 2
Plantear y validar un algoritmo para múltiples detecciones en la imagen "coca_multi.png" con el mismo témplate del ítem 1

In [9]:
# Extension de la funcion multiScaleTemplateMatching para multiples detecciones
def multiScaleTemplateMatchingMultipleDetection(template,image,edge_detector="canny",matchTempMethod = cv.TM_CCOEFF_NORMED,n_scales = 12,threshold=3):
   
    # edge_detector puede ser Canny o Laplacian
    if edge_detector != "canny" and edge_detector != "laplacian":
        print('El parametro edge detector debe ser "canny" o "laplacian"')
        return
    # Paso las imagenes a escala de grises
    imgGray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    tmpGray = cv.cvtColor(template,cv.COLOR_BGR2GRAY)

    # Aplico Gaussian Blur para sacar ruido
    imgGray = cv.GaussianBlur(imgGray,(5,5),0)
    #Obtengo bordes segun el metodo de deteccion
    if edge_detector == "canny":
        imgGray = cv.Canny(imgGray, 100, 120, apertureSize = 3)
    elif edge_detector == "laplacian":
        imgGray = cv.Laplacian(imgGray, cv.CV_8U)
    
    # Voy a reescalar el template n_scales veces partiendo desde el ancho de la imagen
    th,tw = tmpGray.shape
    ih,iw = imgGray.shape
    scales = np.linspace(0, iw/tw, n_scales+1)[1:]

    matching_results = list()
    matching_locations = list()

    confidence_values = list()
        
    for scale in scales:

        # escalo el template
        tmpGray_scaled = cv.resize(tmpGray, None, fx=scale, fy=scale)
        # le aplico el Gaussian Blur para sacar ruido
        tmpGray_scaled = cv.GaussianBlur(tmpGray_scaled,(3,3),0)
        
         #Obtengo bordes segun el metodo de deteccion
        if edge_detector == "canny":
            tmpGray_scaled = cv.Canny(tmpGray_scaled, 150, 180, apertureSize = 3)
        elif edge_detector == "laplacian":
            tmpGray_scaled = cv.Laplacian(tmpGray_scaled, cv.CV_8U)
        # cv.imshow('tmpGray-scaled',tmpGray_scaled)
        # cv.waitKey(0)

        tsh,tsw = tmpGray_scaled.shape

        # Aplico el template matching 
        result = cv.matchTemplate(imgGray, tmpGray_scaled, matchTempMethod)
        # Normalizo el resultado 
        result = (result-np.mean(result))/np.std(result)
        # Guardo el resultado
        matching_results.append(result)
        
        # Guardo los resultados por encima del threshold
        best_results_loc = np.where(result >= threshold)
        for location in zip(*best_results_loc[::-1]): 
            
            matching_locations.append([location[0], location[1], location[0]+tsw, location[1]+tsh])
            
            # Nivel de confianza de la deteccion
            confidence_value = result[location[::-1]]
            confidence_values.append(confidence_value/np.max(result))



    matchImg = image.copy()

    # Aplico NMS para eliminar solapamiento de bounding boxies con un threshold the 0.75
    indices = cv.dnn.NMSBoxes(bboxes=matching_locations, scores=confidence_values, score_threshold=0.75, nms_threshold=0.75)
    for i in indices:
        coord = matching_locations[i]
        
        cv.rectangle(matchImg, (coord[0],coord[1]), (coord[2], coord[3]), (255,0,0), 2)
        
        # La metrica de confianza es medio chamuyo, esta armada para que funcione nms, pero supongo debe haber alguna metrica mejor 
        cv.putText(matchImg,f'Score {str(int(confidence_values[i]*100))}%', (coord[0],coord[1]-6), cv.QT_FONT_NORMAL, 0.45, (255,0,0))


    return matchImg

Probamos la detección del logo con detección de bordes por "Canny" y metodo de detección "Coeficiente de correlación"

Resultado: 18 detecciones sobre 19 totales (94.73%)

In [10]:
scale = 10
threshold = 4.6
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"canny",cv.TM_CCOEFF,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

Probamos la detección del logo con detección de bordes por "Laplaciano" y metodo de detección "Coeficiente de correlación"

Resultado: 12 detecciones sobre 19 totales (63.15%)

In [11]:
scale = 13
threshold = 5.4
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"laplacian",cv.TM_CCOEFF,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

Probamos la detección del logo con detección de bordes por "Canny" y metodo de detección "Correlación Cruzada"

Resultado: 18 detecciones sobre 19 totales (94.73%), con un único solapamiento

In [12]:
scale = 11
threshold = 3.6
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"canny",cv.TM_CCORR,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

Probamos la detección del logo con detección de bordes por "Laplaciano" y metodo de detección "Correlación Cruzada"

Resultado: 19 detecciones sobre 19 totales (100%), con 2 solapamientos 

In [13]:
scale = 13
threshold = 3.9
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi,"laplacian",cv.TM_CCORR,scale,threshold)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()

**Nota: La combinación de (scale;threshold) para los casos anteriores se encontraron de manera iterativa para cada uno de los casos**

## Parte 3

Generalizar el algoritmo del item 2 para todas las imágenes.



Probamos la detección del logo con detección de bordes por "Canny" y metodo de detección "Correlación Cruzada" (multiple detección) y "Correlación Cruzada Normalizada" (detección singular)


In [14]:
# Template Matching usando TM_CCORR_NORMED
edge_detection = "canny" #"canny"
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_1, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_2,edge_detection,cv.TM_CCORR_NORMED, 1,1.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_cola_logo, edge_detection,cv.TM_CCORR_NORMED,10,5.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, logo_1, edge_detection,cv.TM_CCORR_NORMED,10,4.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_1, edge_detection,cv.TM_CCORR_NORMED,12,4.2)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_2, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg) 
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi, edge_detection,cv.TM_CCORR,11,3.6)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()

In [15]:
# Template Matching usando TM_CCORR_NORMED
edge_detection = "laplacian" #"canny"
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_1, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_logo_2,edge_detection,cv.TM_CCORR_NORMED, 1,1.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_cola_logo, edge_detection,cv.TM_CCORR_NORMED,10,4.2)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, logo_1, edge_detection,cv.TM_CCORR_NORMED,10,4.9)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_1, edge_detection,cv.TM_CCORR_NORMED,12,4.2)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_retro_2, edge_detection,cv.TM_CCORR_NORMED,10,5)
cv.imshow('resultImg',resultImg) 
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_multi, edge_detection,cv.TM_CCORR,13,3.9)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)

cv.destroyAllWindows()

Probamos la funcion "multiScaleTemplateMatchingMultipleDetection" con dos imagenes descargas de internet

In [16]:
coca_test = cv.imread('images/coca_test.png')
coca_test_2 = cv.imread('images/coca_test_2.png')
coca_test_3 = cv.imread('images/coca_test_3.png')

edge_detection = "laplacian"
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_test, edge_detection,cv.TM_CCORR,12,5.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_test_2, edge_detection,cv.TM_CCORR,4,3.5)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
resultImg = multiScaleTemplateMatchingMultipleDetection(template, coca_test_3, edge_detection,cv.TM_CCORR,8,5.9)
cv.imshow('resultImg',resultImg)
cv.waitKey(0)
cv.destroyAllWindows()